<a href="https://colab.research.google.com/github/KaterinaKintiDS/Machine_Learning/blob/main/MasterThesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


Μοντέλο πρόβλεψης πιθανότητας σύναψης διακανονισμού

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import re
import datetime
import pickle

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks 
 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from mlxtend.classifier import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn import metrics
from sklearn.metrics import f1_score, classification_report, roc_auc_score, precision_recall_auc_score
from sklearn.metrics import fbeta_score, make_scorer, recall_score, log_loss, confusion_matrix, precision_recall_curve, auc
from sklearn.metrics import accuracy_score, precision_score,  roc_curve,
 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
pd.set_option('display.max_columns', 350)
import warnings
warnings.filterwarnings("ignore")


In [ ]:
path = (r'/content/gdrive/MyDrive/Colab Notebooks/Accounts_Dataset')

list = []

for file_name in os.listdir(path):
    if file_name.endswith(".txt"):
        df = pd.read_csv(os.path.join(path, file_name), header=None, delimiter="\t")
        list.append(df)

df_accounts = pd.concat(list)

In [ ]:
df_accounts.columns = ["IDacc", "SNAPNUMacc" , "MEAS_ACCH_ORIGINATOR" , "MEAS_ACCL_STRAY_PAYMENT_AMT" , "MEAS_ACCL_APLL_IS_NEW" , 
 "MEAS_ACCL_APLL_FORBORNE" , "MEAS_ACCL_APPL_TYPE" , "MEAS_ACCL_PAID_IN_FULL_CM" , "MEAS_ACCL_APPLICATION_SUCCESSFUL" , 
 "MEAS_ACCL_UNDERWRITING_SEGMENT" , "MEAS_ACCL_DPO_OR_REPAYMENT_PLAN" , "MEAS_ACCL_PROMISE_ACCOUNTS" , 
 "MEAS_ACCL_ACTIVATION_PERIOD" , "MEAS_ACCL_PROMISE_ACCOUNT" , "MEAS_ACCL_KEPT_ACCOUNT" , "MEAS_ACCL_KEPT_PROMISE" , 
 "MEAS_ACCL_NUM_OF_PROMISES" , "MEAS_ACCL_HAS_ARRANGEMENT_FLAG" , "MEAS_ACCL_FLAG_RUNNING" , "MEAS_ACCL_APPL_DELINQ_DPD" ,
 "MEAS_ACCL_BROKEN_SETTLEMENT_INDICATOR_CP" , "MEAS_ACCL_ACTL_IS_PHONE_RPC" , "MEAS_ACCL_LMEM_DATE_ENTRY" , "MEAS_ACCL_APPL_STATUS" , 
 "MEAS_ACCL_APPLICATION_BUCKET" , "MEAS_ACCL_LAST_INBOUND_CALL" , "MEAS_ACCL_LAST_OUTBOUND_CALL" , "MEAS_ACAG_DATE_CONTACT_LAST",
 "MEAS_ACCL_LAST_CONTACT_TYPE" , "MEAS_ACCL_LAST_CONTACT_STEP_DATE" , "MEAS_ACCL_LAST_CONTACT_STEP" , "MEAS_ACCL_LAST_CONTACT_STR_DATE" , 
 "MEAS_ACCL_LAST_CONTACT_STRATEGY" , "MEAS_ACCL_LAST_KEPT_BROKEN_DATE" , "MEAS_ACCL_NBR_OF_KEPT_PROMISESS" , "MEAS_ACCL_NBR_OF_KEPT_PROMISES",
 "MEAS_ACCL_NBR_OF_PROMISES" , "MEAS_ACCL_NBR_OF_UNKEPT_PROMISES" , "MEAS_ACCL_REQUEST_STATUS" , "MEAS_ACCL_REQUEST_STATUS_DATE" ,
 "MEAS_ACCH_STATUS_DELINQ_DATE" , "MEAS_ACCH_STATUS_DELINQ" , "MEAS_ACCL_ADMIN_STATUS_EXP_DATE" , "MEAS_ACCL_PREVIOUS_RETURN_DATE" , 
 "MEAS_ACCL_DATE_CAPITALISED" , "MEAS_ACCL_DATE_WRITEOFF_TOBE" , "MEAS_ACCH_DATE_LITIGATION_LATEST" , "MEAS_ACCL_SPECIAL_TREAT_DATE" ,
 "MEAS_ACCH_DATE_DENOUNCED" , "MEAS_ACCH_DATE_DISBURSE_FIRST" , "MEAS_ACCH_STATUS" , "MEAS_ACCL_STATUS_PRIORITY" , 
 "MEAS_ACCL_STATUS_SKIP" , "MEAS_ACCL_BUCKET_MAX_NBG" , "MEAS_ACCH_CATEG_SEGMENT" , "MEAS_ACCL_COOPERATIVE" , "MEAS_ACCH_DATE_OPENED" , 
 "MEAS_ACCH_DATE_BILLING" , "MEAS_ACCL_BILLING_DAY" , "MEAS_ACCT_BRANCH" , "MEAS_ACCH_BUSUNIT" , "MEAS_ACCH_DATE_RESCHEDULING", 
 "MEAS_ACCH_CURRENCY" , "MEAS_ACCH_ACTIVE" , "MEAS_ACCH_DELDAYS" , "MEAS_ACCH_DATE_INSTALMENT_NEXT" , "MEAS_ACCH_DATE_EXP" ,
 "MEAS_ACCL_FIRST_ENTRY_DATE" , "MEAS_ACCL_SECURITIZATION_TYPE" , "MEAS_ACCH_DATE_SECURITISED" , "MEAS_ACCL_DEFAULT_REASON" ,
 "MEAS_ACCL_EBA_STATUS" , "MEAS_ACCH_QC_ISCASE", "MEAS_ACCH_QC_ISCONTRACT" , "MEAS_ACCL_LAST_BILLING_DATE_CLIENT" , 
 "MEAS_ACCH_DELINQ_DATE_EXIT" , "MEAS_ACCH_DELINQ_DATE_ENTRY" , "MEAS_ACCL_LEGAL_ACTION" ,"MEAS_ACCL_LEGAL_ACTION_STATUS",
 "MEAS_ACCH_DATE_BILLING_NEXT" , "MEAS_ACCH_PERIODICITY" , "MEAS_ACCL_RELATION_TYPE_DESC"  , "MEAS_ACCL_RELATION_TYPE" ,
 "MEAS_ACCH_COUNT_RESCHEDULINGS" , "MEAS_ACCL_SOLD_TO" , "MEAS_ACCL_CASE_SOURCE" , "MEAS_ACCH_CATEG_TYPE" , 
 "MEAS_ACCH_TYPE_FORBEARANCE" , "MEAS_ACCH_REASON_NPL" , "MEAS_ACCH_FLAG_LITIGATION" , "MEAS_ACCH_DATE_WRITEOFF" , 
 "MEAS_ACCL_CALL_BACK_COUNTER" , "MEAS_ACCL_MESSAGE_COUNTER" , "MEAS_ACCL_NOT_FOUND_COUNTER" , "MEAS_ACCL_REFUSE_TO_PAY_COUNTER" ,
 "MEAS_ACCL_SECONDARY_FLOW_FLAG" , "MEAS_ACCH_DATE_LITIGATION" , "MEAS_ACCH_STATUS_LITIGATION" ,"MEAS_ACCH_DATE_PAYMENT_LAST", 
 "MEAS_ACCL_LAST_PAYMENT_TYPE" , "MEAS_ACCH_FLAG_COSIGNER" , "MEAS_ACCH_FLAG_GUARANTOR" ,"MEAS_ACCL_NUMBER_OF_COLLATERALS" ,
 "MEAS_ACCL_PAYMENT_IN_ALLOCATION" , "MEAS_ACCH_FLAG_SECURED" , "MEAS_ACCT_CUST_CODE" , "MEAS_ACCL_DEPARTMENT_AT_ALLOCATION",
 "MEAS_ACCL_INDICATION_CONTACT_ANY_CP" , "MEAS_ACCL_INDICATION_CONTACT_ATTEMPT_CP" , "MEAS_ACCL_INDICATION_CONTACT_RIGHTPERSON_CP" ,
 "MEAS_ACCL_MAX_CONTACTTYPE" , "MEAS_ACCL_ALLOCATION_CODE" , "MEAS_ACCH_DELINQ_PACKAGE_DATE" , "MEAS_ACCL_PACKET_DEFINITION_DATE",
 "MEAS_ACCL_PACKET_EXPIRATION_DATE" , "MEAS_ACCH_DELINQ_PACKAGE_TYPE" , "MEAS_PROD_CODE" , "MEAS_ACCL_LAST_RECALL_DATE" , "MEAS_ACCL_REQUEST_CODE" , 
 "MEAS_ACCL_REQUEST_TYPE" , "MEAS_ACCL_DEBT_REDUCTION_RULE" , "MEAS_ACCH_DELINQ_STRATEGY_ENTRYDATE" , 
 "MEAS_ACCH_DELINQ_STEP_ENTRYDATE" , "MEAS_ACCL_LAST_PENDING_ARRANG" , "MEAS_ACCH_DATE_NOMINAL" , "MEAS_ACCH_SNAPNUM" ,
 "MEAS_SPRD_CODE" , "MEAS_ACCH_DELINQ_STRATEGY" , "MEAS_ACCH_DELINQ_STEP" , "MEAS_ACCH_DATE_CLOSED", "MEAS_ACCL_HAS_ARRANGEMENT_NEW" , 
 "MEAS_ACCL_HAS_ARRANGEMENT_CURRENT" , "MEAS_ACCL_DENORM_APLL_DISCOUNT_PERCENTAGE" , "MEAS_ACCL_ARRANGEMENT_DURATION_APPROVED_MAX" , 
 "MEAS_ACCL_DENORM_APLL_STARTING_DATE" , "MEAS_ACCL_ARRANGEMENT_BUCKET_MAX" , "MEAS_ACCL_CONTACT_NUM_ATTEMPTS_CP" , "MEAS_ACCL_CONTACT_NUM_RPC_CP" , 
 "MEAS_ACCL_CONTACT_RPC_TYPE_LATEST" , "MEAS_ACCH_CATEG_CLASS_ASSET" , "MEAS_ACCL_HAS_WEB_ACCOUNT" , "MEAS_ACCL_HAS_PHONE_NUMBER",
 "MEAS_ACCL_HAS_CONTACT_ATTEMPT" , "MEAS_ACCL_FIRST_CONTACT_ATTEMPT" , "MEAS_ACCL_NUM_INSTALMENTS_TOTAL", 
 "MEAS_ACCL_APLL_MATURITY_PERCENTAGE" , "MEAS_ACCL_NR_OF_PAST_UNSETTLED_INSTALLMENTS" , "MEAS_ACCL_APLL_LAST_BROKEN_DATE" , 
 "MEAS_ACCL_DATE_STATUS_REJECTED" , "MEAS_ACCL_APLL_DAYS_PENDING" , "MEAS_ACCL_APLL_DAYS_REJECTED" , "MEAS_ACCL_APLL_DAYS_CANCELLED" , 
 "MEAS_ACCL_APLL_NUM_INSTALMENTS_PAST" , "MEAS_ACCL_LAST_SUBMISSION_DATE" , "MEAS_ACCL_SUBMISSION_AFTER_CONTACT" , 
 "MEAS_ACCL_APPLICATION_PENDING" , "MEAS_ACCL_APPLICATION_ACTIVE_BUCKET_1" , "MEAS_ACCL_APPLICATION_ACTIVE_BUCKET_2" ,
 "MEAS_ACCL_APPLICATION_OPEN" , "MEAS_ACCL_APPLICATION_OPEN_PM" , "MEAS_ACCL_DPD_GROUP" , "MEAS_ACCL_BUS_SIZE" , 
 "MEAS_ACCL_PUBLIC_SECTOR" , "MEAS_ACCL_FLAG_DENOUNCED" , "MEAS_ACCL_EBA_FORBORNE" , "MEAS_ACCL_NUMBER_OF_GUARANTORS" ,
 "MEAS_ACCL_LETTER_FIRST", "MEAS_ACCL_LETTER_SECOND" , "MEAS_ACCL_HAS_FIRST_LETTER" , "MEAS_ACCL_HAS_SECOND_LETTER" , 
 "MEAS_ACCL_LETTER_RECEIVED" , "MEAS_ACCL_LETTER_RECEIVED_DATE" , "MEAS_ACCL_FIRST_CONTACT_ATTEMPT_NM" , "MEAS_ACCL_BROKEN_BUCKET_4" , 
 "MEAS_ACCL_ACTUAL_CONTACTS" , "MEAS_ACCL_ACTUAL_CONTACTS_PM" , "MEAS_ACCL_HAS_COLLATERAL" , "MEAS_ACCL_PAID_IN_FULL" , 
 "MEAS_ACCL_LAST_SETTLEMENT_BUCKET", "MEAS_ACCL_LAST_SETTLEMENT_STATUS" , "MEAS_ACCL_PAYMENT_RECONCILIATION_CODE" , 
 "MEAS_ACCL_DATE_UPDATE_LATEST" , "MEAS_ACCL_FLAG_OPEN_RECENTLY" , "MEAS_ACCL_PHONE_RPC_COUNT_EVER" , "MEAS_ACCL_PHONE_RPC_DATE_FIRST" , 
 "MEAS_ACCL_PHONE_RPC_DATE_LATEST" , "MEAS_ACCL_EED_PERCENT" , "MEAS_ACCH_DELINQ_DCA" , "MEAS_ACCL_PHONE_RPC_DCA_FIRST" , 
 "MEAS_ACCL_PHONE_RPC_DCA_LATEST" , "MEAS_ACCL_DELINQ_DCA_GROUPING" , "MEAS_ACCL_PREVIOUS_3RD_PARTY_NAME" , 
 "MEAS_ACCL_RUNNING_INSTALMENTS_AMT" , "MEAS_ACCL_PAYMENTS_ONEOFF_CP" , "MEAS_ACCL_PAYMENTS_REPAYMENT_PLAN_CP" ,"MEAS_ACCL_PAYMENTS_ONEOFF_YTD" , 
 "MEAS_ACCL_PAYMENTS_REPAYMENT_PLAN_YTD" , "MEAS_ACCL_PAYMENTS_NO_SETTLEMENT_YTD" , "MEAS_ACCL_PAYMENTS_ARRANGEMENTS" , 
 "MEAS_ACCL_AMT_PROMISE" , "MEAS_ACCL_AMT_KEPT" , "MEAS_ACCL_ARRANGEMENT_PRINCIPAL" , "MEAS_ACCL_ARRANGEMENT_BALANCE" , 
 "MEAS_ACCL_AMT_INSTAL_NOT_MATURED_CM" , "MEAS_ACCL_APPL_AMT_APPROVED" , "MEAS_ACCL_APPL_AMT_DUNNING" , 
 "MEAS_ACCL_APPL_AMT_APPROVED_NEW" , "MEAS_ACCL_APPL_AMT_FALLOFF" , "MEAS_ACCL_LMEM_PAYMENTS_CP" , "MEAS_ACCL_LMEM_PAYMENTS_CUMUL" , 
 "MEAS_ACCH_AMT_PAYMENT_CP" , "MEAS_ACCL_AMT_PAYMENTS_TOTAL" , "MEAS_ACCL_AMT_OVERPAYMENT_ADDITIONAL" , "MEAS_ACCL_AMT_PAYMENT_YTD" , 
 "MEAS_ACCH_AMT_PASTDUE" , "MEAS_ACCL_AMT_DENOUNCED" , "MEAS_ACCL_AMT_DUE_NBG" , "MEAS_ACCL_AMT_WRITEOFF_TOBE" , 
 "MEAS_ACCL_AMT_PAYMENT_REMAINING" , "MEAS_ACCH_AMT_BAL_ACCOUNTING" , "MEAS_ACCL_AMT_ORIG_EXPENSES" , 
 "MEAS_ACCL_AMT_ORIG_INTEREST_OFFBALANCE" , "MEAS_ACCL_AMT_ORIG_INTEREST_ONBALANCE" , "MEAS_ACCL_AMT_ORIG_PRINCIPAL" , 
 "MEAS_ACCL_AMT_ORIG_CLAIM_PNI" , "MEAS_ACCL_AMT_OVERPAYMENT" , "MEAS_ACCH_AMT_EXPENSES" , "MEAS_ACCL_AMT_INTEREST_OFFBALANCE" ,
 "MEAS_ACCL_AMT_INTEREST_ONBALANCE" , "MEAS_ACCH_AMT_PRINCIPAL" , "MEAS_ACCL_AMT_CLAIM_PNI" , "MEAS_ACCL_AMT_CLAIM_G3", 
 "MEAS_ACCL_DUE_PRINCIPAL" , "MEAS_ACCL_AMT_CLAIM_G4" , "MEAS_ACCL_AMT_PRINCIPAL_TOTAL" , "MEAS_ACCL_AMT_GBV" , "MEAS_ACCH_AMT_LIMIT" , 
 "MEAS_ACCL_DEFAULT_BALANCE" , "MEAS_ACCH_AMT_ARREARS" , "MEAS_ACCL_LATE_INTEREST" , "MEAS_ACCL_INTEREST" , "MEAS_ACCH_AMT_MINPAYMENT" , 
 "MEAS_ACCH_AMT_BALANCE" , "MEAS_ACCL_PURCHASE_BALANCE" , "MEAS_ACCL_COURT_CLAIMED_AMOUNT" , "MEAS_ACCH_AMT_WRITEOFF" , 
 "MEAS_ACCH_AMT_PAYMENT_LAST" , "MEAS_ACCL_COLLATERALS_VALUE" , "MEAS_ACCL_AMT_INSTALMENTS_CM" , "MEAS_ACCL_AMT_INSTALMENTS_MISSED" , 
 "MEAS_ACCL_AMT_INSTALMENTS_NM" , "MEAS_ACCL_AMT_INSTALMENTS_FUTURE" , "MEAS_ACCL_APPL_AMT_INSTALMENT_NEXT" , "MEAS_ACCL_APLL_AMT_REMAINING" ,
 "MEAS_ACCL_APLL_AMT_WRITEOFF" , "MEAS_ACCL_APLL_AMT_INSTALMENTS_PAST" , "MEAS_ACCL_COLLA_MARKET_VALUE" , "MEAS_ACCL_AMT_BAL_ACCOUNTING_RR" , 
 "MEAS_ACCL_AMT_BAL_OFFBALANCE_RR" , "MEAS_ACCL_AMT_PAYMENTS_EXPECTED_3M", "MEAS_ACCL_AMT_INSTALMENTS_PAID",
 "MEAS_ACCL_COLLATERAL_OR_RE_MARKET_VALUE" , "MEAS_ACCL_EED_GUARANTEED_AMOUNT"]

In [ ]:
df_accounts.head(2)

,IDacc,SNAPNUMacc,MEAS_ACCH_ORIGINATOR,MEAS_ACCL_STRAY_PAYMENT_AMT,MEAS_ACCL_APLL_IS_NEW,MEAS_ACCL_APLL_FORBORNE,MEAS_ACCL_APPL_TYPE,MEAS_ACCL_PAID_IN_FULL_CM,MEAS_ACCL_APPLICATION_SUCCESSFUL,MEAS_ACCL_UNDERWRITING_SEGMENT,MEAS_ACCL_DPO_OR_REPAYMENT_PLAN,MEAS_ACCL_PROMISE_ACCOUNTS,MEAS_ACCL_ACTIVATION_PERIOD,MEAS_ACCL_PROMISE_ACCOUNT,MEAS_ACCL_KEPT_ACCOUNT,MEAS_ACCL_KEPT_PROMISE,MEAS_ACCL_NUM_OF_PROMISES,MEAS_ACCL_HAS_ARRANGEMENT_FLAG,MEAS_ACCL_FLAG_RUNNING,MEAS_ACCL_APPL_DELINQ_DPD,MEAS_ACCL_BROKEN_SETTLEMENT_INDICATOR_CP,MEAS_ACCL_ACTL_IS_PHONE_RPC,MEAS_ACCL_LMEM_DATE_ENTRY,MEAS_ACCL_APPL_STATUS,MEAS_ACCL_APPLICATION_BUCKET,MEAS_ACCL_LAST_INBOUND_CALL,MEAS_ACCL_LAST_OUTBOUND_CALL,MEAS_ACAG_DATE_CONTACT_LAST,MEAS_ACCL_LAST_CONTACT_TYPE,MEAS_ACCL_LAST_CONTACT_STEP_DATE,MEAS_ACCL_LAST_CONTACT_STEP,MEAS_ACCL_LAST_CONTACT_STR_DATE,MEAS_ACCL_LAST_CONTACT_STRATEGY,MEAS_ACCL_LAST_KEPT_BROKEN_DATE,MEAS_ACCL_NBR_OF_KEPT_PROMISESS,MEAS_ACCL_NBR_OF_KEPT_PROMISES,MEAS_ACCL_NBR_OF_PROMISES,MEAS_ACCL_NBR_OF_UNKEPT_PROMISES,MEAS_ACCL_REQUEST_STATUS,MEAS_ACCL_REQUEST_STATUS_DATE,MEAS_ACCH_STATUS_DELINQ_DATE,MEAS_ACCH_STATUS_DELINQ,MEAS_ACCL_ADMIN_STATUS_EXP_DATE,MEAS_ACCL_PREVIOUS_RETURN_DATE,MEAS_ACCL_DATE_CAPITALISED,MEAS_ACCL_DATE_WRITEOFF_TOBE,MEAS_ACCH_DATE_LITIGATION_LATEST,MEAS_ACCL_SPECIAL_TREAT_DATE,MEAS_ACCH_DATE_DENOUNCED,MEAS_ACCH_DATE_DISBURSE_FIRST,MEAS_ACCH_STATUS,MEAS_ACCL_STATUS_PRIORITY,MEAS_ACCL_STATUS_SKIP,MEAS_ACCL_BUCKET_MAX_NBG,MEAS_ACCH_CATEG_SEGMENT,MEAS_ACCL_COOPERATIVE,MEAS_ACCH_DATE_OPENED,MEAS_ACCH_DATE_BILLING,MEAS_ACCL_BILLING_DAY,MEAS_ACCT_BRANCH,MEAS_ACCH_BUSUNIT,MEAS_ACCH_DATE_RESCHEDULING,MEAS_ACCH_CURRENCY,MEAS_ACCH_ACTIVE,MEAS_ACCH_DELDAYS,MEAS_ACCH_DATE_INSTALMENT_NEXT,MEAS_ACCH_DATE_EXP,MEAS_ACCL_FIRST_ENTRY_DATE,MEAS_ACCL_SECURITIZATION_TYPE,MEAS_ACCH_DATE_SECURITISED,MEAS_ACCL_DEFAULT_REASON,MEAS_ACCL_EBA_STATUS,MEAS_ACCH_QC_ISCASE,MEAS_ACCH_QC_ISCONTRACT,MEAS_ACCL_LAST_BILLING_DATE_CLIENT,MEAS_ACCH_DELINQ_DATE_EXIT,MEAS_ACCH_DELINQ_DATE_ENTRY,MEAS_ACCL_LEGAL_ACTION,MEAS_ACCL_LEGAL_ACTION_STATUS,MEAS_ACCH_DATE_BILLING_NEXT,MEAS_ACCH_PERIODICITY,MEAS_ACCL_RELATION_TYPE_DESC,MEAS_ACCL_RELATION_TYPE,MEAS_ACCH_COUNT_RESCHEDULINGS,MEAS_ACCL_SOLD_TO,MEAS_ACCL_CASE_SOURCE,MEAS_ACCH_CATEG_TYPE,MEAS_ACCH_TYPE_FORBEARANCE,MEAS_ACCH_REASON_NPL,MEAS_ACCH_FLAG_LITIGATION,MEAS_ACCH_DATE_WRITEOFF,MEAS_ACCL_CALL_BACK_COUNTER,MEAS_ACCL_MESSAGE_COUNTER,MEAS_ACCL_NOT_FOUND_COUNTER,MEAS_ACCL_REFUSE_TO_PAY_COUNTER,MEAS_ACCL_SECONDARY_FLOW_FLAG,MEAS_ACCH_DATE_LITIGATION,MEAS_ACCH_STATUS_LITIGATION,MEAS_ACCH_DATE_PAYMENT_LAST,MEAS_ACCL_LAST_PAYMENT_TYPE,MEAS_ACCH_FLAG_COSIGNER,MEAS_ACCH_FLAG_GUARANTOR,MEAS_ACCL_NUMBER_OF_COLLATERALS,MEAS_ACCL_PAYMENT_IN_ALLOCATION,MEAS_ACCH_FLAG_SECURED,MEAS_ACCT_CUST_CODE,MEAS_ACCL_DEPARTMENT_AT_ALLOCATION,MEAS_ACCL_INDICATION_CONTACT_ANY_CP,MEAS_ACCL_INDICATION_CONTACT_ATTEMPT_CP,MEAS_ACCL_INDICATION_CONTACT_RIGHTPERSON_CP,MEAS_ACCL_MAX_CONTACTTYPE,MEAS_ACCL_ALLOCATION_CODE,MEAS_ACCH_DELINQ_PACKAGE_DATE,MEAS_ACCL_PACKET_DEFINITION_DATE,MEAS_ACCL_PACKET_EXPIRATION_DATE,MEAS_ACCH_DELINQ_PACKAGE_TYPE,MEAS_PROD_CODE,MEAS_ACCL_LAST_RECALL_DATE,MEAS_ACCL_REQUEST_CODE,MEAS_ACCL_REQUEST_TYPE,MEAS_ACCL_DEBT_REDUCTION_RULE,MEAS_ACCH_DELINQ_STRATEGY_ENTRYDATE,MEAS_ACCH_DELINQ_STEP_ENTRYDATE,MEAS_ACCL_LAST_PENDING_ARRANG,MEAS_ACCH_DATE_NOMINAL,MEAS_ACCH_SNAPNUM,MEAS_SPRD_CODE,MEAS_ACCH_DELINQ_STRATEGY,MEAS_ACCH_DELINQ_STEP,MEAS_ACCH_DATE_CLOSED,MEAS_ACCL_HAS_ARRANGEMENT_NEW,MEAS_ACCL_HAS_ARRANGEMENT_CURRENT,MEAS_ACCL_DENORM_APLL_DISCOUNT_PERCENTAGE,MEAS_ACCL_ARRANGEMENT_DURATION_APPROVED_MAX,MEAS_ACCL_DENORM_APLL_STARTING_DATE,MEAS_ACCL_ARRANGEMENT_BUCKET_MAX,MEAS_ACCL_CONTACT_NUM_ATTEMPTS_CP,MEAS_ACCL_CONTACT_NUM_RPC_CP,MEAS_ACCL_CONTACT_RPC_TYPE_LATEST,MEAS_ACCH_CATEG_CLASS_ASSET,MEAS_ACCL_HAS_WEB_ACCOUNT,MEAS_ACCL_HAS_PHONE_NUMBER,MEAS_ACCL_HAS_CONTACT_ATTEMPT,MEAS_ACCL_FIRST_CONTACT_ATTEMPT,MEAS_ACCL_NUM_INSTALMENTS_TOTAL,MEAS_ACCL_APLL_MATURITY_PERCENTAGE,MEAS_ACCL_NR_OF_PAST_UNSETTLED_INSTALLME

In [ ]:
df_accounts.shape

(4074754, 263)

In [ ]:
df_accounts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4074754 entries, 0 to 342399
Columns: 263 entries, IDacc to MEAS_ACCL_EED_GUARANTEED_AMOUNT
dtypes: float64(134), int64(27), object(102)
memory usage: 8.0+ GB


In [ ]:
path = (r"/content/gdrive/MyDrive/Colab Notebooks/Customers_Dataset")

list = []

for file_name in os.listdir(path):
    if file_name.endswith(".txt"):
        df = pd.read_csv(os.path.join(path, file_name), header=None, delimiter="\t")
        list.append(df)

df_customers = pd.concat(list)

In [ ]:
df_customers.columns = ['IDcus', 'SNAPNUMcus', 'MEAS_CUSH_CATEG_SEGMENT', 'MEAS_CUSL_HAS_CONTACT_ATTEMPT_EVER', 'MEAS_CUSL_PROFESSION_GROUPING',
'MEAS_CUSL_PHONE_RPC_COUNT', 'MEAS_CUSL_COUNT_RPC', 'MEAS_CUSL_COUNT_NO_CONTACT', 'MEAS_CUSL_REFUSE', 'MEAS_CUSL_PROMISE', 
'MEAS_CUSL_COMPLAIN', 'MEAS_CUSL_COMMUNICATION_ATTEMPT', 'MEAS_CUSL_MIN_COMMUNICATION_DATE_3M', 'MEAS_CUSL_RESPONDED', 
'MEAS_CUSL_COUNT_3RD_PARTY', 'MEAS_CUSL_POSTAL_REGION', 'MEAS_CUSL_PREFECTURE', 'MEAS_CUSL_GEOGRAPHICAL_REGION', 
'MEAS_CUSL_MAINLAND_ISLAND', 'MEAS_CUSL_MAX_PERSON_CONTACT_TYPE', 'MEAS_CUSL_DIALLER_ATTEMPTS_COUNT', 'MEAS_CUSL_HELLO_LETTER_RESULT',
'MEAS_CUSL_CITY', 'MEAS_CUSL_COUNTRY' , 'MEAS_CUSH_ADDR_PRIMARY_REGION' , 'MEAS_CUSL_ADMINISTRATION_STATUS_DATE' , 
'MEAS_CUSL_ADMINISTRATION_STATUS', 'MEAS_CUSL_HAS_ACTIVE_ARRANGEMENT' , 'MEAS_CUSL_HAS_PAYMENT', 'MEAS_CUSL_PRIORITY_CUSTOMER_STATUS_DATE', 
'MEAS_CUSH_STATUS_LITIGATION ' , 'MEAS_CUSH_COUNT_PHONES', 'MEAS_CUSL_COLLECTION_CASES_NUM' , 'MEAS_CUSL_MAX_CURRENT_BUCKET' , 
'MEAS_CUSL_MIN_CURRENT_BUCKET' , 'MEAS_CUSL_HAS_RESTRUCTURING_APPLICATION' , 'MEAS_CUSH_OWNER_HOME' , 'MEAS_CUSL_IS_CO_OWNER' , 
'MEAS_CUSL_IS_GUARANTOR' , 'MEAS_CUSL_IS_PRIMARY_OWNER' , 'MEAS_CUSL_LAST_PAYMENT_DATE' , 'MEAS_CUSL_LAST_DEFAULT_DATE' , 
'MEAS_CUSL_FIRST_DEFAULT_DATE', 'MEAS_CUSL_COLLECTION_CASES_NUM_PRIMARY_OWNER' , 'MEAS_CUSL_MAX_CURRENT_BUCKET_PRIMARY_OWNER' , 
'MEAS_CUSL_TOTAL_ACCOUNTS' , 'MEAS_CUSH_BIRTHDATE' , 'MEAS_CUSH_ID_TYPE' , 'MEAS_CUSH_FAMSTATUS' , 'MEAS_CUSH_PROFESSION', 
'MEAS_CUSL_POSITION', 'MEAS_CUSH_GENDER' , 'MEAS_CUSL_STATUS_LAW3869' , 'MEAS_CUSH_DATE_NOMINAL' , 'MEAS_CUSH_STATUS', 
'MEAS_CUSL_HAS_CONTACT_ATTEMPT' , 'MEAS_CUSL_LAST_PAYMENT_AMOUNT', 'MEAS_CUSL_DEBT_AMOUNT_PRIMARY_OWNER' , 'MEAS_CUSL_TOTAL_OUTSTANDING_AMOUNT' , 
'MEAS_CUSL_DEBT_AMOUNT' , 'MEAS_CUSL_ACCOUNTING_BALANCE' , 'MEAS_CUSL_BALANCE', 'MEAS_CUSL_BALANCE_PRIMARY_OWNER']

In [ ]:
df_customers[:3]

,IDcus,SNAPNUMcus,MEAS_CUSH_CATEG_SEGMENT,MEAS_CUSL_HAS_CONTACT_ATTEMPT_EVER,MEAS_CUSL_PROFESSION_GROUPING,MEAS_CUSL_PHONE_RPC_COUNT,MEAS_CUSL_COUNT_RPC,MEAS_CUSL_COUNT_NO_CONTACT,MEAS_CUSL_REFUSE,MEAS_CUSL_PROMISE,MEAS_CUSL_COMPLAIN,MEAS_CUSL_COMMUNICATION_ATTEMPT,MEAS_CUSL_MIN_COMMUNICATION_DATE_3M,MEAS_CUSL_RESPONDED,MEAS_CUSL_COUNT_3RD_PARTY,MEAS_CUSL_POSTAL_REGION,MEAS_CUSL_PREFECTURE,MEAS_CUSL_GEOGRAPHICAL_REGION,MEAS_CUSL_MAINLAND_ISLAND,MEAS_CUSL_MAX_PERSON_CONTACT_TYPE,MEAS_CUSL_DIALLER_ATTEMPTS_COUNT,MEAS_CUSL_HELLO_LETTER_RESULT,MEAS_CUSL_CITY,MEAS_CUSL_COUNTRY,MEAS_CUSH_ADDR_PRIMARY_REGION,MEAS_CUSL_ADMINISTRATION_STATUS_DATE,MEAS_CUSL_ADMINISTRATION_STATUS,MEAS_CUSL_HAS_ACTIVE_ARRANGEMENT,MEAS_CUSL_HAS_PAYMENT,MEAS_CUSL_PRIORITY_CUSTOMER_STATUS_DATE,MEAS_CUSH_STATUS_LITIGATION,MEAS_CUSH_COUNT_PHONES,MEAS_CUSL_COLLECTION_CASES_NUM,MEAS_CUSL_MAX_CURRENT_BUCKET,MEAS_CUSL_MIN_CURRENT_BUCKET,MEAS_CUSL_HAS_RESTRUCTURING_APPLICATION,MEAS_CUSH_OWNER_HOME,MEAS_CUSL_IS_CO_OWNER,MEAS_CUSL_IS_GUARANTOR,MEAS_CUSL_IS_PRIMARY_OWNER,MEAS_CUSL_LAST_PAYMENT_DATE,MEAS_CUSL_LAST_DEFAULT_DATE,MEAS_CUSL_FIRST_DEFAULT_DATE,MEAS_CUSL_COLLECTION_CASES_NUM_PRIMARY_OWNER,MEAS_CUSL_MAX_CURRENT_BUCKET_PRIMARY_OWNER,MEAS_CUSL_TOTAL_ACCOUNTS,MEAS_CUSH_BIRTHDATE,MEAS_CUSH_ID_TYPE,MEAS_CUSH_FAMSTATUS,MEAS_CUSH_PROFESSION,MEAS_CUSL_POSITION,MEAS_CUSH_GENDER,MEAS_CUSL_STATUS_LAW3869,MEAS_CUSH_DATE_NOMINAL,MEAS_CUSH_STATUS,MEAS_CUSL_HAS_CONTACT_ATTEMPT,MEAS_CUSL_LAST_PAYMENT_AMOUNT,MEAS_CUSL_DEBT_AMOUNT_PRIMARY_OWNER,MEAS_CUSL_TOTAL_OUTSTANDING_AMOUNT,MEAS_CUSL_DEBT_AMOUNT,MEAS_CUSL_ACCOUNTING_BALANCE,MEAS_CUSL_BALANCE,MEAS_CUSL_BALANCE_PRIMARY_OWNER
0,219460,37,NaN,Yes,NaN,NaN,NaN,NaN,No,No,No,No,2018-11-07,No,NaN,Υπόλοιπο Δυτικής Αττικής και Ανατολική Αττική,Αττικής,Στερεά Ελλάδα,Mainland,No Contact,0,Absent,ΝΕΑ ΜΑΚΡΗ,NaN,ΚΑΤΟΙΚΙΑΣ,2018-07-01 00:00:00.000,ZERO PHONES,0,1,2018-07-01,ΑΦΑΙΡΕΣΗ ΝΟΜΗΣ,0,4,Bucket 13,Bucket 13,0,0,0,0,1,2007-05-18,2009-02-22,2006-03-31,4,Bucket 13,4,1949-05-26,ΑΣΤΥΝΟΜΙΚΗ ΤΑΥΤΟΤΗΤΑ,ΑΓΑΜΟΣ,ΠΟΛΙΤΙΚΟΣ ΜΗΧΑΝΙΚΟΣ,ΦΥΣΙΚΟ ΠΡΟΣΩΠΟ,M,NaN,2019-01-01,ΟΡΙΣΤΙΚΗ ΔΙΑΓΡΑΦΗ,No,407.3940,6359.9758,6461.2506,6402.8720,5119.1890,6384.3673,6347.7831
1,217109,37,NaN,Yes,NaN,0.0,0.0,1.0,No,No,No,No,NaN,No,0.0,Νότια Προάστια Θεσσαλονίκης και Τριανδρία,Θεσσαλονίκης,Μακεδονία,Mainland,No Contact,0,Unknown,ΚΑΛΑΜΑΡΙΑ,NaN,ΕΠΙΚΟΙΝΩΝΙΑΣ,2018-07-01 00:00:00.000,ZERO PHONES,0,0,NaN,NaN,0,1,Bucket 13,Bucket 13,0,0,0,0,1,NaN,NaN,NaN,1,Bucket 13,1,1957-01-01,ΑΣΤΥΝΟΜΙΚΗ ΤΑΥΤΟΤΗΤΑ,ΑΓΝΩΣΤΟ,ΛΟΙΠΟΙ ΥΠΑΛΛΗΛΟΙ,ΦΥΣΙΚΟ ΠΡΟΣΩΠΟ,M,NaN,2019-01-01,ΟΡΙΣΤΙΚΗ ΔΙΑΓΡΑΦΗ,No,0.0000,7465.7155,7584.5978,7516.0696,2881.3937,7494.3477,7451.4029
2,219461,37,NaN,Yes,NaN,0.0,0.0,1.0,No,No,No,No,2018-11-01,No,0.0,Πάτρα,Αχαΐας,Πελοπόννησος,Mainland,No Contact,0,Unknown,ΠΑΤΡΑ,NaN,ΕΠΙΚΟΙΝΩΝΙΑΣ,2018-07-01 00:00:00.000,FOR CONTACT,0,1,NaN,NaN,1,1,Bucket 13,Bucket 13,0,0,0,0,1,2008-05-13,2008-04-06,2008-04-06,1,Bucket 13,1,1956-06-05,ΔΙΑΒΑΤΗΡΙΟ,ΕΓΓΑΜΟΣ,ΛΟΙΠΟΙ,ΦΥΣΙΚΟ ΠΡΟΣΩΠΟ,M,NaN,2019-01-01,ΟΡΙΣΤΙΚΗ ΔΙΑΓΡΑΦΗ,Yes,153.2514,2999.8742,3047.6435,3020.1075,758.9547,3011.3792,2994.1231


In [ ]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2368272 entries, 0 to 197355
Data columns (total 63 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   IDcus                                         int64  
 1   SNAPNUMcus                                    int64  
 2   MEAS_CUSH_CATEG_SEGMENT                       object 
 3   MEAS_CUSL_HAS_CONTACT_ATTEMPT_EVER            object 
 4   MEAS_CUSL_PROFESSION_GROUPING                 object 
 5   MEAS_CUSL_PHONE_RPC_COUNT                     float64
 6   MEAS_CUSL_COUNT_RPC                           float64
 7   MEAS_CUSL_COUNT_NO_CONTACT                    float64
 8   MEAS_CUSL_REFUSE                              object 
 9   MEAS_CUSL_PROMISE                             object 
 10  MEAS_CUSL_COMPLAIN                            object 
 11  MEAS_CUSL_COMMUNICATION_ATTEMPT               object 
 12  MEAS_CUSL_MIN_COMMUNICATION_DATE_3M           object 
 13

In [ ]:
df_customers.shape

(2368272, 63)

Merge

In [ ]:
mergeAccCus = pd.merge(df_accounts,df_customers,left_on = ['MEAS_ACCT_CUST_CODE', 'SNAPNUMacc'],right_on = ['IDcus', 'SNAPNUMcus'], how = 'inner')

In [ ]:
mergeAccCus[:3]

,IDacc,SNAPNUMacc,MEAS_ACCH_ORIGINATOR,MEAS_ACCL_STRAY_PAYMENT_AMT,MEAS_ACCL_APLL_IS_NEW,MEAS_ACCL_APLL_FORBORNE,MEAS_ACCL_APPL_TYPE,MEAS_ACCL_PAID_IN_FULL_CM,MEAS_ACCL_APPLICATION_SUCCESSFUL,MEAS_ACCL_UNDERWRITING_SEGMENT,MEAS_ACCL_DPO_OR_REPAYMENT_PLAN,MEAS_ACCL_PROMISE_ACCOUNTS,MEAS_ACCL_ACTIVATION_PERIOD,MEAS_ACCL_PROMISE_ACCOUNT,MEAS_ACCL_KEPT_ACCOUNT,MEAS_ACCL_KEPT_PROMISE,MEAS_ACCL_NUM_OF_PROMISES,MEAS_ACCL_HAS_ARRANGEMENT_FLAG,MEAS_ACCL_FLAG_RUNNING,MEAS_ACCL_APPL_DELINQ_DPD,MEAS_ACCL_BROKEN_SETTLEMENT_INDICATOR_CP,MEAS_ACCL_ACTL_IS_PHONE_RPC,MEAS_ACCL_LMEM_DATE_ENTRY,MEAS_ACCL_APPL_STATUS,MEAS_ACCL_APPLICATION_BUCKET,MEAS_ACCL_LAST_INBOUND_CALL,MEAS_ACCL_LAST_OUTBOUND_CALL,MEAS_ACAG_DATE_CONTACT_LAST,MEAS_ACCL_LAST_CONTACT_TYPE,MEAS_ACCL_LAST_CONTACT_STEP_DATE,MEAS_ACCL_LAST_CONTACT_STEP,MEAS_ACCL_LAST_CONTACT_STR_DATE,MEAS_ACCL_LAST_CONTACT_STRATEGY,MEAS_ACCL_LAST_KEPT_BROKEN_DATE,MEAS_ACCL_NBR_OF_KEPT_PROMISESS,MEAS_ACCL_NBR_OF_KEPT_PROMISES,MEAS_ACCL_NBR_OF_PROMISES,MEAS_ACCL_NBR_OF_UNKEPT_PROMISES,MEAS_ACCL_REQUEST_STATUS,MEAS_ACCL_REQUEST_STATUS_DATE,MEAS_ACCH_STATUS_DELINQ_DATE,MEAS_ACCH_STATUS_DELINQ,MEAS_ACCL_ADMIN_STATUS_EXP_DATE,MEAS_ACCL_PREVIOUS_RETURN_DATE,MEAS_ACCL_DATE_CAPITALISED,MEAS_ACCL_DATE_WRITEOFF_TOBE,MEAS_ACCH_DATE_LITIGATION_LATEST,MEAS_ACCL_SPECIAL_TREAT_DATE,MEAS_ACCH_DATE_DENOUNCED,MEAS_ACCH_DATE_DISBURSE_FIRST,MEAS_ACCH_STATUS,MEAS_ACCL_STATUS_PRIORITY,MEAS_ACCL_STATUS_SKIP,MEAS_ACCL_BUCKET_MAX_NBG,MEAS_ACCH_CATEG_SEGMENT,MEAS_ACCL_COOPERATIVE,MEAS_ACCH_DATE_OPENED,MEAS_ACCH_DATE_BILLING,MEAS_ACCL_BILLING_DAY,MEAS_ACCT_BRANCH,MEAS_ACCH_BUSUNIT,MEAS_ACCH_DATE_RESCHEDULING,MEAS_ACCH_CURRENCY,MEAS_ACCH_ACTIVE,MEAS_ACCH_DELDAYS,MEAS_ACCH_DATE_INSTALMENT_NEXT,MEAS_ACCH_DATE_EXP,MEAS_ACCL_FIRST_ENTRY_DATE,MEAS_ACCL_SECURITIZATION_TYPE,MEAS_ACCH_DATE_SECURITISED,MEAS_ACCL_DEFAULT_REASON,MEAS_ACCL_EBA_STATUS,MEAS_ACCH_QC_ISCASE,MEAS_ACCH_QC_ISCONTRACT,MEAS_ACCL_LAST_BILLING_DATE_CLIENT,MEAS_ACCH_DELINQ_DATE_EXIT,MEAS_ACCH_DELINQ_DATE_ENTRY,MEAS_ACCL_LEGAL_ACTION,MEAS_ACCL_LEGAL_ACTION_STATUS,MEAS_ACCH_DATE_BILLING_NEXT,MEAS_ACCH_PERIODICITY,MEAS_ACCL_RELATION_TYPE_DESC,MEAS_ACCL_RELATION_TYPE,MEAS_ACCH_COUNT_RESCHEDULINGS,MEAS_ACCL_SOLD_TO,MEAS_ACCL_CASE_SOURCE,MEAS_ACCH_CATEG_TYPE,MEAS_ACCH_TYPE_FORBEARANCE,MEAS_ACCH_REASON_NPL,MEAS_ACCH_FLAG_LITIGATION,MEAS_ACCH_DATE_WRITEOFF,MEAS_ACCL_CALL_BACK_COUNTER,MEAS_ACCL_MESSAGE_COUNTER,MEAS_ACCL_NOT_FOUND_COUNTER,MEAS_ACCL_REFUSE_TO_PAY_COUNTER,MEAS_ACCL_SECONDARY_FLOW_FLAG,MEAS_ACCH_DATE_LITIGATION,MEAS_ACCH_STATUS_LITIGATION,MEAS_ACCH_DATE_PAYMENT_LAST,MEAS_ACCL_LAST_PAYMENT_TYPE,MEAS_ACCH_FLAG_COSIGNER,MEAS_ACCH_FLAG_GUARANTOR,MEAS_ACCL_NUMBER_OF_COLLATERALS,MEAS_ACCL_PAYMENT_IN_ALLOCATION,MEAS_ACCH_FLAG_SECURED,MEAS_ACCT_CUST_CODE,MEAS_ACCL_DEPARTMENT_AT_ALLOCATION,MEAS_ACCL_INDICATION_CONTACT_ANY_CP,MEAS_ACCL_INDICATION_CONTACT_ATTEMPT_CP,MEAS_ACCL_INDICATION_CONTACT_RIGHTPERSON_CP,MEAS_ACCL_MAX_CONTACTTYPE,MEAS_ACCL_ALLOCATION_CODE,MEAS_ACCH_DELINQ_PACKAGE_DATE,MEAS_ACCL_PACKET_DEFINITION_DATE,MEAS_ACCL_PACKET_EXPIRATION_DATE,MEAS_ACCH_DELINQ_PACKAGE_TYPE,MEAS_PROD_CODE,MEAS_ACCL_LAST_RECALL_DATE,MEAS_ACCL_REQUEST_CODE,MEAS_ACCL_REQUEST_TYPE,MEAS_ACCL_DEBT_REDUCTION_RULE,MEAS_ACCH_DELINQ_STRATEGY_ENTRYDATE,MEAS_ACCH_DELINQ_STEP_ENTRYDATE,MEAS_ACCL_LAST_PENDING_ARRANG,MEAS_ACCH_DATE_NOMINAL,MEAS_ACCH_SNAPNUM,MEAS_SPRD_CODE,MEAS_ACCH_DELINQ_STRATEGY,MEAS_ACCH_DELINQ_STEP,MEAS_ACCH_DATE_CLOSED,MEAS_ACCL_HAS_ARRANGEMENT_NEW,MEAS_ACCL_HAS_ARRANGEMENT_CURRENT,MEAS_ACCL_DENORM_APLL_DISCOUNT_PERCENTAGE,MEAS_ACCL_ARRANGEMENT_DURATION_APPROVED_MAX,MEAS_ACCL_DENORM_APLL_STARTING_DATE,MEAS_ACCL_ARRANGEMENT_BUCKET_MAX,MEAS_ACCL_CONTACT_NUM_ATTEMPTS_CP,MEAS_ACCL_CONTACT_NUM_RPC_CP,MEAS_ACCL_CONTACT_RPC_TYPE_LATEST,MEAS_ACCH_CATEG_CLASS_ASSET,MEAS_ACCL_HAS_WEB_ACCOUNT,MEAS_ACCL_HAS_PHONE_NUMBER,MEAS_ACCL_HAS_CONTACT_ATTEMPT,MEAS_ACCL_FIRST_CONTACT_ATTEMPT,MEAS_ACCL_NUM_INSTALMENTS_TOTAL,MEAS_ACCL_APLL_MATURITY_PERCENTAGE,MEAS_ACCL_NR_OF_PAST_UNSETTLED_INSTALLME

In [ ]:
mergeAccCus.shape

(4074754, 326)

New Dataset με Λογαριασμούς χωρίς ενεργό, επιτυχημένο ή εν αναμονή διακανονισμό, στους οποίους έχει γίνει πρόσφατη προσπάθεια επικοινωνίας

In [ ]:
mergeAccCus['MEAS_CUSL_MIN_COMMUNICATION_DATE_3M'] = pd.to_datetime(mergeAccCus['MEAS_CUSL_MIN_COMMUNICATION_DATE_3M'])
mergeAccCus['MEAS_ACCH_DATE_NOMINAL'] = pd.to_datetime(mergeAccCus['MEAS_ACCH_DATE_NOMINAL'])

In [ ]:
df = mergeAccCus.loc[
    (mergeAccCus['MEAS_ACCL_APPLICATION_PENDING'].isnull()) | (mergeAccCus['MEAS_ACCL_APPLICATION_PENDING'] == 'No')
    & (mergeAccCus['MEAS_ACCL_APPL_STATUS'].isnull() | ~mergeAccCus['MEAS_ACCL_APPL_STATUS'].isin(['Approved', 'Running', 'Fulfilled', 'Partially Fulfilled', 'Out of collection']))
]


df = df.reset_index(drop=True)

del mergeAccCus

In [ ]:
df['MEAS_ACCL_PAID_IN_FULL_CM'].value_counts()

In [ ]:
df['MEAS_ACCL_APPL_STATUS'].value_counts()

In [ ]:
df.shape

Target Variable

Υπολογισμός της πιθανότητας σύναψης διακανονισμού και πληρωμής μιας δόσης μέσα στους επόμενους 2 μήνες

In [ ]:
# Target variable == True for all rows where the paid is YES and the account has made a payment in the current month or the following two months.
df['target'] = False

snapnumacc_yes = df['MEAS_ACCL_PAID_IN_FULL'] == 'Yes'
snapnumacc = df[snapnumacc_yes]['SNAPNUMacc']
snapnumacc_idxs = ((df['SNAPNUMacc'].isin(snapnumacc)) | (df['SNAPNUMacc'].isin(snapnumacc + 1)) | (df['SNAPNUMacc'].isin(snapnumacc + 2)))
df.loc[snapnumacc_yes & snapnumacc_idxs, 'target'] = True

In [ ]:
df[:3]

In [ ]:
df['target'].value_counts()

In [ ]:
df.info()

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing.head(25)

In [ ]:

missing_percentages = df.isna().sum() / len(df) * 100
cols_high_missing = missing_percentages[missing_percentages > 90].index
print(cols_high_missing)


In [ ]:
df = df.drop(cols_high_missing, axis=1)

In [ ]:
df.shape

From the categorical values, lets see first the dates

In [ ]:
categ_dates = df.filter(regex='DATE', axis=1)
categ_dates

In [ ]:
# Convert the date columns to datetime format
categ_dates = categ_dates.apply(pd.to_datetime, errors='coerce')
df[categ_dates.columns] = categ_dates

df['MEAS_CUSL_ADMINISTRATION_STATUS_DATE'].dtypes

In [ ]:
#change the format to "YYYY-mm-dd
for col in categ_dates:    
    if df[col].dtype == 'object' and ' 00:00:00' in df[col].iloc[0]:      
        df[col] = pd.to_datetime(df[col]).dt.date.astype(str)

df['MEAS_CUSL_ADMINISTRATION_STATUS_DATE'].value_counts()

Categorical Values

In [ ]:
categ = df.select_dtypes(include=['object']).columns

In [ ]:
#finding null values in categorical columns
null_mask = df[categ].isnull()
null_counts = null_mask.sum()
print(null_counts)

In [ ]:
# class 1 = minority clss
df['target'].value_counts()

In [ ]:
df['MEAS_ACCL_LAST_CONTACT_TYPE'].unique()

In [ ]:
df['MEAS_ACCL_LAST_CONTACT_TYPE'].isnull().sum()



In [ ]:
#Fill NanValues with No contact
df['MEAS_ACCL_LAST_CONTACT_TYPE'].fillna('No Contact',inplace = True)

In [ ]:
df['MEAS_ACCL_NBR_OF_KEPT_PROMISES'].fillna('undefined',inplace = True)

In [ ]:
df['MEAS_ACCL_STATUS_SKIP'].isnull().sum()


In [ ]:
df[df['target'] == 1]['MEAS_ACCL_STATUS_SKIP'].isna()

In [ ]:
df['MEAS_ACCL_STATUS_SKIP'].fillna('not mentioned',inplace = True)

In [ ]:
df['MEAS_ACCH_PERIODICITY'].fillna('RANDOM INTERVAL',inplace = True)

In [ ]:
df['MEAS_ACCL_CONTACT_RPC_TYPE_LATEST'].fillna('Undefined',inplace = True)

In [ ]:
for col in categ:
    print(df[col].value_counts())
    print('-'*20) 

In [ ]:
df = df.reset_index(drop=True)
df[:3]

In [ ]:
# fill categorical values with NV
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace(['', ' ', None], 'NV')
        

In [ ]:
df[:5]

Numerical columns

In [ ]:
numer = df.select_dtypes(include=[np.number])
numer.info()

In [ ]:
numer.describe()

In [ ]:
#left skewed

for col in numer:
    print(col)
    print('Skew :',round(df[col].skew(),2))
    plt.figure(figsize=(15,4))
    plt.subplot(1,2,1)
    df[col].hist(bins=10, grid=False)
    plt.ylabel('count')
    plt.subplot(1,2,2)
    sns.boxplot(x=df[col])
    plt.show()
    

In [ ]:
#fill numerical values with 9696
for col in df.columns:
    if df[col].dtype == 'int' or df[col].dtype == 'float':
        df[col] = df[col].fillna('9696')

In [ ]:
df.isnull().sum()

In [ ]:
df[:15]

In [ ]:
# stratified sampling
sam = fint.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.05))

In [ ]:
sam=sam.reset_index(drop=True)

In [ ]:
sam.shape

In [ ]:
corr_matrix = sam.corr().abs()

In [ ]:
# get upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

In [ ]:
# find features with correlation greater than 0.95
correlated = [column for column in upper.columns if any(upper[column] > 0.95)]
correlated

In [ ]:
# drop highly correlated features
sam.drop([ 'MEAS_ACCL_ARRANGEMENT_BALANCE',
 'MEAS_ACCL_AMT_PAYMENT_YTD',   
 'MEAS_ACCH_AMT_EXPENSES',  
 'MEAS_ACCL_AMT_INTEREST_ONBALANCE',  
 'MEAS_ACCL_AMT_CLAIM_PNI',
 'MEAS_ACCL_AMT_PRINCIPAL_TOTAL',
 'MEAS_ACCL_AMT_GBV',
 'MEAS_ACCH_AMT_ARREARS', 
 'MEAS_ACCL_INTEREST',
 'MEAS_ACCH_AMT_BALANCE',
 'MEAS_ACCL_PURCHASE_BALANCE',
 'MEAS_ACCL_AMT_BAL_ACCOUNTING_RR', 
 'MEAS_CUSL_COLLECTION_CASES_NUM_PRIMARY_OWNER',  
 'MEAS_CUSL_TOTAL_OUTSTANDING_AMOUNT', 
 'MEAS_CUSL_BALANCE_PRIMARY_OWNER'], axis=1, inplace=True) 


In [ ]:
#Dates turned into numerical
#dates subtracted by 2018-01-01 
date_cols = [col for col in sam.columns if pd.api.types.is_datetime64_dtype(sam[col])]

for col in date_cols:
    sam[col] = ((sam[col] - pd.Timestamp('2018-01-01')) / pd.Timedelta('30 days')).round(2)

sam = sam.rename(columns={col: col+'_months_since_2018' for col in date_cols})


In [ ]:
# Replace infinite values with NaN
sam = sam.replace([np.inf, -np.inf], np.nan)

# Fill NaN values with 969
sam = sam.fillna(969)

In [ ]:
long_values = ["MEAS_ACCH_DELINQ_STRATEGY", "MEAS_ACCL_FIRST_CONTACT_ATTEMPT", "MEAS_ACCL_LETTER_FIRST", 'MEAS_ACCL_LETTER_SECOND', "MEAS_CUSL_PROFESSION_GROUPING", 'MEAS_CUSL_POSTAL_REGION', "MEAS_CUSL_PREFECTURE", "MEAS_CUSH_ADDR_PRIMARY_REGION", "MEAS_CUSL_CITY", "MEAS_CUSH_ID_TYPE"]
sam = sam.drop(long_values, axis=1)

In [ ]:
sam.shape

In [ ]:
# Replace True with 1 and False with 0 in the target column
sam['target'] = sam['target'].replace({True: 1, False: 0})

# Count the number of 1s and 0s in the target column
count_ones = (sam['target'] == 1).sum()
count_zeros = (sam['target'] == 0).sum()


print("Number of 0s in target column:", count_zeros)
print("Number of 1s in target column:", count_ones)


In [ ]:
# calculate percentage of each class of target variable
class_counts = sam['target'].value_counts(normalize=True) * 100
print(class_counts)

In [ ]:
# replace the special characters of categorical columns
categ = sam.select_dtypes(include=['object']).columns
for col in categ:
    sam[col] = sam[col].str.replace('[%/_\-,.:&€=+\'">]', ' ', regex=True)


In [ ]:
sam['MEAS_ACCL_APPL_TYPE'].unique()

In [ ]:
cat_cols = sam.select_dtypes(include=['object']).columns

# Convert categorical values to strings
sam[cat_cols] = sam[cat_cols].astype(str)
encoded_sam = pd.DataFrame()

 
for col in cat_cols:
    encoded_sam = pd.get_dummies(sam, columns=cat_cols)
    
sam = pd.concat([sam.drop(cat_cols, axis=1), encoded_sam], axis=1)


In [ ]:

sam[:3]

In [ ]:
X = df.drop('target', axis=1).values
y = df['target'].values

Logistic Regression

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

oversampler = SMOTE(sampling_strategy="minority")

LR = LogisticRegression(random_state=16, C=3,  class_weight='balanced', n_jobs=-1)


In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

 
for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    #scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Oversample the minority class   
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)
    
    
    LR.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    
    
    print(classification_report(y_test, y_pred))

In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
LR.fit(X_train_resampled, y_train_resampled)
y_pred = LR.predict(X_test_scaled)
y_train_prob = LR.predict_proba(X_train_resampled)[:, 1]
y_test_prob = LR.predict_proba(X_test_scaled)[:, 1]
y_train_pred = LR.predict(X_train_resampled)


In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['Logistic Regression'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve LogisticRegression (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

Random Forest

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

oversampler = SMOTE(sampling_strategy='minority')
RF = RandomForestClassifier(class_weight='balanced', n_jobs=-1) # n_estimators = 10, criterion = ‘entropy’, random_state = 0

In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

 
for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    #scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Oversample the minority class   
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)
    
      
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    
    
    print(classification_report(y_test, y_pred))

In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
RF.fit(X_train_resampled, y_train_resampled)
y_pred = RF.predict(X_test_scaled)
y_train_prob = RF.predict_proba(X_train_resampled)[:, 1]
y_test_prob = RF.predict_proba(X_test_scaled)[:, 1]
y_train_pred = RF.predict(X_train_resampled)


In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['Random Forest'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve Random Forest (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
# Calculate feature importances
feature_names = list(df.columns[:-1])
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
names_sorted = [feature_names[i] for i in indices]

# Plot the top 20 features
num_features = 20
plt.figure()
plt.title("Top 20 Feature Importance")
plt.bar(range(num_features), importances[indices][:num_features])
plt.xticks(range(num_features), names_sorted[:num_features], rotation=90)
plt.show()


SVM

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

oversampler = SMOTE(sampling_strategy='minority') 

 
SVM = SVC(alpha=0.0001, class_weight='balanced', n_jobs=-1)

In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Scaling
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)
    
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    
    print(classification_report(y_test, y_pred))


In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
SVM.fit(X_train_resampled, y_train_resampled)
y_pred = model.predict(X_test_scaled)
y_train_prob = model.predict_proba(X_train_resampled)[:, 1]
y_test_prob = model.predict_proba(X_test_scaled)[:, 1]
y_train_pred = model.predict(X_train_resampled)

In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['SVC'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve SVM (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

ADABoost

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

#undersampler =RandomUnderSampler(sampling_strategy=0.5)#,'majority'
oversampler = SMOTE(sampling_strategy='minority') #, adasyn,sampling_strategy=0.1

AB = AdaBoostClassifier(n_estimators=50, learning_rate=1)

In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

 
for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    #scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Oversample the majority class   
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)
    
    # Undersample the minority class
    #X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)
    
    #X_train_resampled, y_train_resampled = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
    #X_train_resampled, y_train_resampled = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))
    
    
    AB.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    
    
    print(classification_report(y_test, y_pred))

In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
AB.fit(X_train_resampled, y_train_resampled)
y_pred = model.predict(X_test_scaled)
y_train_prob = model.predict_proba(X_train_resampled)[:, 1]
y_test_prob = model.predict_proba(X_test_scaled)[:, 1]
y_train_pred = model.predict(X_train_resampled)


In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['AdaBoost'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve AdaBoost (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

XGBoost

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

oversampler = SMOTE(sampling_strategy='minority')

XGB = XGBClassifier(class_weight='balanced', n_jobs=-1)

In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

 
for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    #Scaling
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)
    
    
    XGB.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    
    
    print(classification_report(y_test, y_pred))

In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
XGB.fit(X_train_resampled, y_train_resampled)
y_pred = XGB.predict(X_test_scaled)
y_train_prob = XGB.predict_proba(X_train_resampled)[:, 1]
y_test_prob = XGB.predict_proba(X_test_scaled)[:, 1]
y_train_pred = XGB.predict(X_train_resampled)


In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['XGBoost'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve XGBoost (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
Stacking

In [ ]:
#Stacking Classifier base:LR,RF,XGB  meta-learner: logistic regression
model_1 = LogisticRegression(C=3, class_weight='balanced', n_jobs=-1)
model_1.fit(train, y_train)
clf_1 = CalibratedClassifierCV(model_1, method='sigmoid')

model_2 = RandomForestClassifier(n_estimators=250, max_depth=25, class_weight='balanced', n_jobs=-1)
model_2.fit(train, y_train)
clf_2 = CalibratedClassifierCV(model_2, method='sigmoid')

model_3 = XGBClassifier(max_depth=5, n_estimators=100, class_weight='balanced', n_jobs=-1)
model_3.fit(train, y_train)
clf_3 = CalibratedClassifierCV(model_3, method='sigmoid')

stack_clf = StackingClassifier(classifiers=[clf_1, clf_2, clf_3], meta_classifier=LogisticRegression(), use_probas=True)
stack_clf.fit(train, y_train)

In [ ]:
# predict probabilities 
y_probs_train = stack_clf.predict_proba(train)[:, 1]
y_probs_test = stack_clf.predict_proba(test)[:, 1]
y_predicted_train = stack_clf.predict(train)
y_predicted_test = stack_clf.predict(test)
train_auc = roc_auc_score(y_train, y_probs_train)
test_auc = roc_auc_score(y_test, y_probs_test)
train_acc = accuracy_score(y_train, y_predicted_train)
test_acc = accuracy_score(y_test, y_predicted_test)

In [ ]:
#Accuracy and AUC
print('*'*50)
print('Train AUC: %.3f' % train_auc)
print('Test AUC: %.3f' % test_auc)
print('*'*50)
print('Train Accuracy: %.3f' % train_acc)
print('Test Accuracy: %.3f' % test_acc)

# Plot ROC curve
train_fpr, train_tpr, _ = roc_curve(y_train, y_probs_train)
test_fpr, test_tpr, _ = roc_curve(y_test, y_probs_test)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
#Train Confusion Matrix
from sklearn.metrics import confusion_matrix

cma = confusion_matrix(y_train, y_predicted_train)
print('Confusion matrix:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve Stacking (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

Voting(soft)

In [ ]:
# create individual models
model_1 = LogisticRegression(C=3, class_weight='balanced', n_jobs=-1)
model_2 = RandomForestClassifier(n_estimators=250, max_depth=25, class_weight='balanced', n_jobs=-1)
model_3 = XGBClassifier(max_depth=5, n_estimators=100 ,class_weight='balanced', n_jobs=-1)

# create calibrated classifiers for each model
clf_1 = CalibratedClassifierCV(model_1, method='sigmoid')
clf_2 = CalibratedClassifierCV(model_2, method='sigmoid')
clf_3 = CalibratedClassifierCV(model_3, method='sigmoid')

# create stacking classifier with best hyperparameter for meta-classifier
log_reg = LogisticRegression(C=0.0001, n_jobs=-1)
stack_clf = StackingClassifier(classifiers=[clf_1, clf_2, clf_3], meta_classifier=log_reg, use_probas=True)
stack_clf.fit(train, y_train)

# create voting classifier with soft voting
model = VotingClassifier(estimators=[('log_reg', clf_1), ('rf', model_2), ('stack', stack_clf), ('xgb', model_3)], voting='soft')
model.fit(train, y_train)

y_probs_train = model.predict_proba(train)
y_probs_test = model.predict_proba(test)
y_predicted_train = model.predict(train)
y_predicted_test = model.predict(test)

# keep probabilities for the positive outcome only
y_probs_train = y_probs_train[:, 1]
y_probs_test = y_probs_test[:, 1]


In [ ]:
# calculate AUC and Accuracy
train_auc = roc_auc_score(y_train, y_probs_train)
test_auc = roc_auc_score(y_test, y_probs_test)
train_acc = accuracy_score(y_train, y_predicted_train)
test_acc = accuracy_score(y_test, y_predicted_test)

In [ ]:
print('*'*50)
print('Train AUC: %.3f' % train_auc)
print('Test AUC: %.3f' % test_auc)
print('*'*50)
print('Train Accuracy: %.3f' % train_acc)
print('Test Accuracy: %.3f' % test_acc)
score['Voting Classifier'] = [test_auc, test_acc]

In [ ]:
# calculate roc curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train, y_probs_train)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_probs_test)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
#Train Confusion Matrix

from sklearn.metrics import confusion_matrix

cma = confusion_matrix(y_train, y_predicted_train)
print('Confusion matrix:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt='g')


In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve Voting (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
print('***************  Comparison of different models  ****************')
table = PrettyTable(['Model', 'ROC AUC', 'Precision-Recall AUC'])
for item in score.items():
    model_name = item[0]
    roc_auc = item[1][0]
    precision_recall_auc = precision_recall_auc_score(y_test, model.predict_proba(X_test)[:,1])
    table.add_row([model_name, roc_auc, precision_recall_auc])
print(table)


In [ ]:
from sklearn.metrics import precision_recall_curve, auc

# calculate precision-recall AUC scores for each model
pr_auc_scores = {}
for model_name, (y_pred_train, y_prob_train, y_pred_test, y_prob_test) in predictions.items():
    precision, recall, _ = precision_recall_curve(y_test, y_prob_test)
    pr_auc_scores[model_name] = auc(recall, precision)

# create pretty table for comparison of precision-recall AUC scores
pr_auc_table = PrettyTable(['Model', 'Precision-Recall AUC'])
for model_name, pr_auc_score in pr_auc_scores.items():
    pr_auc_table.add_row([model_name, pr_auc_score])

print('***************  Comparison of Precision-Recall AUC Scores  ****************')
print(pr_auc_table)
